In [1]:
import os, tqdm, time, random
import cv2 as cv
import numpy as np
import pandas as pd
import nibabel as nib
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import models, layers, losses, optimizers

from loss import *
from utils import *
from network import *

random.seed(777)
tf.set_random_seed(777)
np.random.seed(777)

In [2]:
root_dir = '../data/for_section/'

fold_idx = 1

train_path = os.path.join(root_dir, 'for_fold_%d'%fold_idx)

val_path = os.path.join(root_dir, 'for_fold_%d_val'%fold_idx)

In [24]:
date_lists = sorted(os.listdir('./checkpoint/'))
print(date_lists)
selected_date_idx = int(input("Please Select ckpt"))

ckpt_lists = sorted(os.listdir('./checkpoint/'+date_lists[selected_date_idx]))
print(ckpt_lists)

selected_ckpt_idx = int(input("Please Select ckpt"))

ckpt_root = './checkpoint/%s/%s'%(date_lists[selected_date_idx], ckpt_lists[selected_idx])
result_root = './result/%s/%s'%(date_lists[selected_date_idx], ckpt_lists[selected_idx])

print(ckpt_root)
print(result_root)
   
weights_lists = [i for i in sorted(os.listdir(ckpt_root)) if '.h5' in i]

with open(os.path.join(ckpt_root, "model.json"), "r") as json_file:
    model_json = json_file.read()

G = models.model_from_json(model_json)    
G.load_weights(os.path.join(ckpt_root, weights_lists[-1]))
G = models.Model(inputs = G.input, outputs = G.outputs[0])
print("\nModel Loaded!\n")

['.ipynb_checkpoints', '20190916_3DSR.h5', 'Sep_16_2019', 'Sep_17_2019', 'Sep_19_2019', 'Sep_22_2019', 'Sep_23_2019', 'Sep_27_2019', 'Sep_30_2019']


Please Select ckpt 4


['DeepSR_msegrad_test', 'RBSR64_msegrad_test', 'RBSRGAN64_mse_test']


KeyboardInterrupt: 

In [20]:
# For Gil
scan_list = sorted(os.listdir(val_path))[1:]
for scan in scan_list:
    dante_path = os.path.join(val_path, scan, 'T1SPACE09mmISOPOSTwDANTE')
    img_name = [i for i in os.listdir(dante_path) if '.nii' in i and '_rsl' not in i][0]
    #print(img_name)
    val = nib.load(os.path.join(dante_path, img_name))
    val = check_data(val.get_data())
    
test_in = []
cor, sag, axi = val.shape
tmp = np.array(np.dsplit(val, axi//6))
tmp = tmp.mean(axis=-1)
tmp = np.transpose(tmp, [1, 2, 0])

#half_top = int(np.floor(tmp.shape[-1]/2))
#half_bot = int(np.ceil(tmp.shape[-1]/2))

recon = np.zeros(shape=[cor, sag, axi])

slice_dict = {
    1:[[0, 160], [0, 160], [0, 128], [0, 128]], 
    2:[[0, 160], [96, 256], [0, 128], [32, 256]], 
    3:[[96, 256], [0, 160], [32, 256], [0, 128]],
    4:[[96, 256], [96, 256], [32, 256], [32, 256]]
}
test = {}
sli = 12
i = 0
for row in range(2):
    row_start = row*128
    for col in range(2):
        
        col_start = col*128
        print(i, row_start, col_start)
        test[i] = G.predict(tmp[np.newaxis, 
                                   slice_dict[i+1][0][0]:slice_dict[i+1][0][1],
                                   slice_dict[i+1][1][0]:slice_dict[i+1][1][1],
                                   :, np.newaxis])
        recon[row_start:row_start+128, 
              col_start:col_start+128] = test[i][0,
                                                  slice_dict[i+1][2][0]:slice_dict[i+1][2][1],
                                                  slice_dict[i+1][3][0]:slice_dict[i+1][3][1], :, 0]
        i += 1
# test_3d_2 = net.predict(tmp[np.newaxis, ..., half_top:, np.newaxis])

aff = np.eye(4)
aff[2, 2]=6

nib.save(nib.Nifti1Image(tmp, aff), os.path.join(result_root, 'val_input.nii'))
nib.save(nib.Nifti1Image(recon, np.eye(4)), os.path.join(result_root, 'val_pred.nii'))
nib.save(nib.Nifti1Image(val, np.eye(4)), os.path.join(result_root, 'val_label.nii'))

0 0 0
1 0 128
2 128 0
3 128 128


In [9]:
scan_list = sorted(os.listdir(val_path))[1:]
for scan in scan_list:
    dante_path = os.path.join(val_path, scan, 'T1SPACE09mmISOPOSTwDANTE')
    img_name = [i for i in os.listdir(dante_path) if '.nii' in i and '_rsl' not in i][0]
    #print(img_name)
    val = nib.load(os.path.join(dante_path, img_name))
    val = check_data(val.get_data())
    
val_in = []
cor, sag, axi = val.shape
tmp = np.array(np.dsplit(val, axi//6))
tmp = tmp.mean(axis=-1)
tmp = np.transpose(tmp, [1, 2, 0])

#half_top = int(np.floor(tmp.shape[-1]/2))
#half_bot = int(np.ceil(tmp.shape[-1]/2))

In [10]:
recon = np.zeros(shape=[cor, sag, axi])

test = {}
sli = 12
for i in range(0, tmp.shape[-1], sli):
    print(i)
    test['test_3d_%d'%i] = net.predict(tmp[np.newaxis, ..., i:i+sli, np.newaxis])
    recon[..., i*6:(i+sli)*6] = test['test_3d_%d'%i][0, ..., 0]
# test_3d_2 = net.predict(tmp[np.newaxis, ..., half_top:, np.newaxis])

0
12


In [11]:
aff = np.eye(4)
aff[2, 2]=6

nib.save(nib.Nifti1Image(tmp, aff), os.path.join(result_root, 'val_input.nii'))
nib.save(nib.Nifti1Image(recon, np.eye(4)), os.path.join(result_root, 'val_pred.nii'))
nib.save(nib.Nifti1Image(val, np.eye(4)), os.path.join(result_root, 'val_label.nii'))